In [4]:
!pip install scikit-learn
!pip install rasterio

In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
train_files = ['tile00.csv', 'tile01.csv', 'tile03.csv']
train = pd.concat([pd.read_csv(f) for f in train_files ], ignore_index=True)
X_train = train.loc[:, train.columns != 'class']
y_train = train[['class']]



In [7]:
# Function to train a Random Forest classifier
def train_random_forest(X, y):
    clf = RandomForestClassifier(n_estimators=10, random_state=42, verbose=2)
    clf.fit(X, y)
    return clf

# Function to evaluate the classifier on a test set
def evaluate_classifier(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return y_pred, accuracy, report


In [4]:
X_train = X_train.rename(columns={x:y for x,y in zip(X_train.columns,range(0,len(X_train.columns)))})
X_train.fillna(0, inplace=True)
y_train.fillna(0, inplace=True)

/tmp/ipykernel_15522/2697274735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.fillna(0, inplace=True)


In [5]:
# Train the Random Forest classifier
classifier = train_random_forest(X_train, y_train.values.ravel())


/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


In [8]:

test = pd.read_csv('tile02.csv')
X_test = test.loc[:, test.columns != 'class']
y_test = test[['class']]
# Evaluate the classifier on the test set
y_pred, accuracy, report = evaluate_classifier(classifier, X_test, y_test)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9997643236317042
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   6117118
           1       0.00      0.00      0.00       636
           2       0.00      0.00      0.00       488
           4       0.00      0.00      0.00       144
           5       0.00      0.00      0.00       122
           6       0.00      0.00      0.00        51
           7       0.00      0.00      0.00         1

    accuracy                           1.00   6118560
   macro avg       0.14      0.14      0.14   6118560
weighted avg       1.00      1.00      1.00   6118560



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test.values.ravel(), y_pred, average='weighted',labels=np.unique(y_pred))

In [ ]:
set(y_test.values.ravel()) - set(y_pred)

## Hyperparamter Tuning with GridSearch

In [23]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [3, 6, 9], 
    'max_leaf_nodes': [3, 6, 9], 
} 
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)


grid_search.fit(X_train, y_train.values.ravel())
grid_search.best_params_

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=25; total time= 1.1min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=25; total time= 1.2min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=25; total time= 1.3min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=50; total time= 2.1min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=50; total time= 2.2min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=50; total time= 2.7min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=100; total time= 4.2min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=100; total time= 4.4min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=3, n_estimators=100; total time= 4.9min
[CV] END max_depth=3, max_features=sqrt, max_leaf_nodes=6, n_estimators=25; tot

KeyboardInterrupt: 